In [4]:
import keras_tuner
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import time
import os

In [5]:
def Mobilenet_v2(input_size,weights,Dropout_rate,Trainable,alpha = 0.35):
    base_model = keras.applications.MobileNetV2(
        input_shape=(input_size, input_size, 3),
        alpha=alpha,
        weights=weights,
        include_top=False
    )
    inputs = keras.Input(shape=(input_size, input_size,3))

    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    x = scale_layer(inputs)
    x = base_model(x, training=False)
    if Trainable:
        base_model.trainable =True
    else:
        base_model.trainable = False
        print("特征层已冻结")
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(Dropout_rate, name='Dropout')(x)
    outputs = keras.layers.Dense(15, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    return model


class HyperMobilenet_v2(keras_tuner.HyperModel):
    def __init__(self,input_size,batch_size,train_root):
        super(keras_tuner.HyperModel, self).__init__()
        self.input_size = input_size
        self.batch_size = batch_size
        self.train_root = train_root
    def model(self,Dropout_rate,lr):
        model = Mobilenet_v2(
            input_size=self.input_size,
            weights='imagenet',
            Dropout_rate=Dropout_rate,
            Trainable=True
        )
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss=keras.losses.CategoricalCrossentropy(),
                    metrics=["accuracy"])
        return model
    def build(self, hp):
        lr = hp.Float("lr",min_value=1e-5,max_value=1e-3,sampling="log")
        Dropout_rate = hp.Choice("Dropout_rate",values =[0.1,0.2,0.3,0.4,0.5,0.6])
        return self.model(Dropout_rate,lr) 
    
    def dataset(self, hp):

        train_root = self.train_root

        zoom_range = hp.Float("zoom_range",min_value=0.1,max_value=0.3,sampling="linear")
        # channel_shift_range = hp.Int("channel_shift_range",min_value=0,max_value=30,step=10)

        train_generator = ImageDataGenerator(rotation_range=360,
                                            zoom_range =zoom_range,
                                            horizontal_flip = True,
                                            validation_split =0.2,
                                            channel_shift_range =channel_shift_range
                                            )
        train_dataset = train_generator.flow_from_directory(batch_size=self.batch_size,
                                                            directory=train_root,
                                                            shuffle=True,
                                                            target_size=(self.input_size,self.input_size),
                                                            subset='training')
        valid_dataset = train_generator.flow_from_directory(batch_size=self.batch_size,
                                                            directory=train_root,
                                                            shuffle=True,
                                                            target_size=(self.input_size,self.input_size),
                                                            subset='validation')
        return train_dataset,valid_dataset

    def fit(self, hp, model,**kwargs):
        
        train_dataset,valid_dataset = self.dataset(hp)

        return model.fit(
            train_dataset,
            validation_data=valid_dataset,
            **kwargs
        ) 

In [6]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel =HyperMobilenet_v2(input_size=128,batch_size=128,train_root='/root/autodl-tmp/AllAlphaBets'),
    objective="val_accuracy",
    max_trials=50,
    overwrite=True,
    directory="my_dir",
    project_name="tune_hypermodel",
)
tuner.search(epochs=10,workers=8)

Trial 2 Complete [00h 00m 08s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 16s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
3.2437e-05        |0.00025926        |lr
0.2               |0.6               |Dropout_rate
0.16549           |0.1               |zoom_range
30                |0                 |channel_shift_range



2024-08-14 02:39:24.550244: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_14 in the registry.
Traceback (most recent call last):

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 258, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_14 in the registry.


2024-08-14 02:39:24.550351: W tensorflow/core/kernels/data/generator_dataset_op.cc:108] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_14 in the registry.
Traceback (most recent call last):

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 258, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_14 in the registry.


	 [[{

Found 64712 images belonging to 15 classes.
Found 16171 images belonging to 15 classes.
Epoch 1/10


2024-08-14 02:39:32.491937: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2024-08-14 02:39:32.491965: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2024-08-14 02:39:32.492047: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1#TC: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/stream_executor/cuda/cuda_dnn.cc(4023): 'cudnnConvolutionForward( cudnn.handle(), alpha, input_nd_.handle(), input_data.opaque(), filter_.handle(), filter_data.opaque(), conv_.handle(), ToConvForwardAlgo(algo), scratch_memory.opaque(), scratch_memo

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/root/miniconda3/lib/python3.8/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/root/miniconda3/lib/python3.8/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/root/miniconda3/lib/python3.8/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/tmp/ipykernel_20631/355277094.py", line 76, in fit
    return model.fit(
  File "/root/miniconda3/lib/python3.8/site-packages/keras/utils/